# 크로스 밸리데이션
    * 지금까지 교제에서 해왔었던 방식은 2분할 방식이였음
        * 훈련전용 데이터와 테스트 전용 데이터로 분할한 뒤 서로 크로스로 학습의 타당성을 검증하는 방법
    * K 분할 교차 검증 ( K-fold cross vaildation )
        * 훈련 데이터를 K개로 나누고 그중에 한묶음을 검증데이터로 사용하고 나머지 묶음을 훈련 데이터로 사용하여 학습을 시킴
![Alt text](https://tensorflowkorea.files.wordpress.com/2016/10/07_cross_validation_diagram.png)


In [19]:
from sklearn import svm, metrics
import random, re

# 붓꽃의 CSV 파일 읽어 들이기 
lines = open('iris.csv', 'r', encoding='utf-8').read().split("\n")

# 문자열 데이터 숫자변환 람다함수 
f_tonum = lambda n : float(n) if re.match(r'^[0-9\.]+$', n) else n
# 쉽표로 데이터 뽑아주는 람다함수 ( 공백제거 후 ',' 스플릿)
f_cols  = lambda li: list(map(f_tonum, li.strip().split(',')))
csv = list(map(f_cols, lines))

# 헤더 제거하기 (epalLength,SepalWidth,PetalLength,PetalWidth,Name)
del csv[0] 

print(type(csv))

<class 'list'>


In [32]:
# 데이터를 K개로 분할하기 

K = 5 
csvk = [ [] for i in range(K) ]
print(len(csvk))
print(type(csvk[0])) 

print(len(csv))
for i in range(len(csv)):
    csvk[i % K].append(csv[i])



5
<class 'list'>
150


In [38]:
# 리스트를 훈련 전용 데이터와 테스트 전용 데이터로 분할하는 함수
def split_data_label(rows):
    data = []; label = []
    for row in rows:
        data.append(row[0:4])
        label.append(row[4])
    return (data, label)

# 정답률 구하는 함수 
def calc_score(test, train):
    test_f, test_l = split_data_label(test)
    train_f, train_l = split_data_label(train)
    # 학습시키고 정답률 구하기
    clf = svm.SVC()
    clf.fit(train_f, train_l)
    pre = clf.predict(test_f)
    return metrics.accuracy_score(test_l, pre)

# K개로 분할해서 정답률 구하기
score_list = []
for testc in csvk:
    # testc 이외의 데이터를 훈련 전용 데이터로 사용하기
    trainc = []
    for i in csvk:
        if i != testc: trainc += i
    sc = calc_score(testc, trainc)
    score_list.append(sc)
    
print("각각의 정답률 =", score_list)
print("평균 정답률 =", sum(score_list) / len(score_list))

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
각각의 정답률 = [0.9666666666666667, 0.9333333333333333, 0.9666666666666667, 0.9666666666666667, 1.0]
평균 정답률 = 0.9666666666666668


# scikit-learn 의 크로스 밸리데이션

* API 
http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html

    * cross_val_score() 함수 사용 및 이해는 소스를 통해서 확인


In [45]:
import pandas as pd
from sklearn import svm, metrics, model_selection
import random, re

# 붓꽃의 CSV 데이터 읽어 들이기 
csv = pd.read_csv('iris.csv')

# 리스트를 훈련 전용 데이터와 테스트 전용 데이터로 분할하기 
data = csv[["SepalLength","SepalWidth","PetalLength","PetalWidth"]]
# print(data)
label = csv["Name"]
# print(label)

# 크로스 밸리데이션하기
clf = svm.SVC()
scores = model_selection.cross_val_score(clf, data, label, cv=5)
print("각각의 정답률 =", scores)
print("평균 정답률 =", scores.mean())

각각의 정답률 = [0.96666667 1.         0.96666667 0.96666667 1.        ]
평균 정답률 = 0.9800000000000001


In [68]:
from sklearn.datasets import load_iris

# scikit-learn 의 붓꽃 데이터셋
# http://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html
iris = load_iris()

clf = svm.SVC()
scores = model_selection.cross_val_score(clf, iris.data, iris.target, cv=5)
print("각각의 정답률 =", scores)
print("평균 정답률 =", scores.mean())


각각의 정답률 = [0.96666667 1.         0.96666667 0.96666667 1.        ]
평균 정답률 = 0.9800000000000001
